In [ ]:
import os

import pandas as pd
import polars as pl

In [ ]:
CWD = os.getcwd()
PATH_INPUT_FOLDER = os.path.join(CWD, "input")

In [ ]:
fs = sorted([os.path.join(PATH_INPUT_FOLDER, f) for f in os.listdir(PATH_INPUT_FOLDER)])

In [ ]:
df = pd.concat([pd.read_parquet(path = path) for path in fs[:5]])

In [ ]:
114_842_782

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.head()["TpepPickupDatetime"].dt.day

In [ ]:
# %%timeit

# (
#     df
#     [["TpepPickupDatetime", "TripDistance", "TotalAmount", "PaymentType"]]
#     .assign(
#         DateOfWeek = df["TpepPickupDatetime"].dt.day
#     )
#     .set_index("TpepPickupDatetime")
#     .groupby(["PaymentType"])
#     .resample("D")
#     ["TripDistance"]
#     .mean()
# )

# ### 4.16 s ± 50.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
4.16 * 12.666666666666666

In [ ]:
114/9

In [ ]:
(
    df
    [["TpepPickupDatetime", "TripDistance", "TotalAmount", "PaymentType"]]
    .assign(
        DateOfWeek = df["TpepPickupDatetime"].dt.day
    )
    .set_index("TpepPickupDatetime")
    .groupby(["PaymentType"])
    .resample("D")
    ["TripDistance"]
    .mean()
)

In [ ]:
os.path.join(PATH_INPUT_FOLDER, f"yellow*.parquet")

In [ ]:
pldf = pl.scan_parquet(source = os.path.join(PATH_INPUT_FOLDER, f"yellow*.parquet"))

In [ ]:
# pandas: 9_344_926   --> 4.16 s --> Todo en Memoria
# polars: 114_842_782 --> 7.99 s --> Leyendo del disco -> procesando en dataset

In [ ]:
pldf.head().collect()

In [ ]:
pldf.select(pl.col("PassengerCount").is_null()).collect().sum()

In [ ]:
plan = (
    pldf
    .with_columns(
        DayOfWeek = pl.col("TpepPickupDatetime").dt.weekday()
    )
    .filter(
        ~pl.col("PassengerCount").is_null()        
    )
    .select(pl.col(["TpepPickupDatetime", "TripDistance", "TotalAmount", "PaymentType"]))
    .sort("TpepPickupDatetime")
    .group_by_dynamic(
        index_column = "TpepPickupDatetime",
        every = "1h",
        group_by = "PaymentType"
    )
    .agg(
        MeanTripDistance = pl.col("TripDistance").mean()
    )
)

In [ ]:
# Polars: Query Engine con API DataFrame

In [ ]:
(
    plan
    .collect()
    .head()
)

In [ ]:
plan

In [ ]:
plan.show_graph(optimized = True)

In [ ]:
# pl.scan_parquet?? # Lazy

In [ ]:
# %%timeit

# (
#     pldf
#     .select(pl.col(["TpepPickupDatetime", "TripDistance", "TotalAmount", "PaymentType"]))
#     .with_columns(
#         DateOfWeek = pl.col("TpepPickupDatetime").dt.weekday()
#     )
#     .sort("TpepPickupDatetime")
#     .group_by_dynamic(
#         index_column = "TpepPickupDatetime",
#         every = "1h",
#         group_by = "PaymentType"
#     )
#     .agg(
#         MeanTripDistance = pl.col("TripDistance").mean()
#     )
#     .collect()
# )

# ### 7.81 s ± 1.44 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
